# DHS grouping
Does further preprocessing steps after the unified columns have been loaded from DHS_loading_from_preprocessing.ipynb
Especially:
- loading locations file and merging them
- groups on either GEID (a survey ID) and adm2 or GEID and cluster ID
- creates DHS dataset

In [224]:
from icecream import ic
import scipy.stats
import pandas as pd
import numpy as np
from warnings import warn

In [225]:
# folder_path = "/mnt/datadisk/data/surveys/DHS_final_raw_data/"
# work_dir = "/mnt/datadisk/data/Projects/water/pickles/"
input_dir = "/mnt/datadisk/data/Projects/water/inputs/"
# min_version = 3
overwrite_pqt = True
dataset_type = 'HR'  #, 'BR', 'CR', 'HW', 'IR', 'KR', 'MR', 'PR', 'AR'] ??['AN', 'FC, 'FP', 'HW', 'PV', 'SC'] < 25 surveys
# 'BR' (birth R), 'HW!' (WHO height/weight standards children), 'IR'/'PR'? (Individual R, hh member R,), 'KR' (children R), 'AR' (HIV R)

urban_rural_all_mode = 'all' # 'all', 'U', 'R'
# 'adm2_gaul' or 'cluster number' (possibly 'adm1_gaul' etc.); additionally to survey ID
group_by_cols = ['Meta; GEID_init', 'Meta; adm2_gaul']
preprocessing_f = f"{input_dir}Manually_adjusted_df_V3_{dataset_type}.pkl"
# unify_columns_f = f"{input_dir}summary_df_V3_{dataset_type}_unify_cols_reviewed.csv"
locations_f = input_dir + 'locations.csv'
# numeric_out_f = f"{input_dir}grouped_df_V3_{dataset_type}_{group_by_col}_numeric{add_str}.pkl"
out_f = f"{input_dir}grouped_df_V3_{dataset_type}_{group_by_cols[1].split('; ')[1]}_{urban_rural_all_mode}.pkl"
meta_data_cols_out_f = f"{input_dir}grouped_df_V3_{dataset_type}_{group_by_cols[1].split('; ')[1]}_meta_data_cols_{urban_rural_all_mode}.pkl"
### to do: urban/rural mode
### 2: drop binary, urban/rural, 2nd column -> in loading, drop binary, urban/rural, 2nd column
#to do: this should be created after grouping!? Maybe it makes sense even before grouping??? 
# -> yes prolly, as it increases the continuity of the distribution (more HH than adm2) (these are not the same if compared for urban/rural modes)
one_hot_cols = ['adm0_gaul']
frequency_hot_cols = ['adm1_gaul', 'adm2_gaul']
# cut_perc = 10

In [226]:
locations_df = pd.read_csv(locations_f)
print(locations_df.columns.to_list())
locations_df = locations_df[['GEID', 'TIF_name', 'adm2_name', 'adm1_name', 'adm0_name', 'DHSCLUST', 'URBAN_RURA']]
locations_df = locations_df.rename(columns={'GEID': 'GEID_init'})
locations_df = locations_df.rename(columns={'TIF_name': 'GEID', 'adm2_name': 'adm2_gaul', 'adm1_name': 'adm1_gaul', 'adm0_name': 'adm0_gaul', 'DHSCLUST': 'DHSCLUST'})
locations_df

['country', 'DHSID', 'DHSCC', 'DHSYEAR', 'DHSCLUST', 'CCFIPS', 'ADM1FIPS', 'ADM1FIPSNA', 'ADM1SALBNA', 'ADM1SALBCO', 'ADM1DHS', 'ADM1NAME', 'DHSREGCO', 'DHSREGNA', 'SOURCE', 'URBAN_RURA', 'LATNUM', 'LONGNUM', 'ALT_GPS', 'ALT_DEM', 'DATUM', 'GEID', 'TIF_name', 'adm2_code', 'adm2_name', 'adm1_code', 'adm1_name', 'adm0_code', 'adm0_name', 'Shape_Area']


,GEID_init,GEID,adm2_gaul,adm1_gaul,adm0_gaul,DHSCLUST,URBAN_RURA
0,AOGE52FL,AOGE52FL00000001,Lobito,Benguela,Angola,1,U
1,AOGE52FL,AOGE52FL00000002,Lobito,Benguela,Angola,2,U
2,AOGE52FL,AOGE52FL00000003,Benguela,Benguela,Angola,3,U
3,AOGE52FL,AOGE52FL00000004,Sumbe,Cuanza Sul,Angola,4,U
4,AOGE52FL,AOGE52FL00000005,Dande,Bengo,Angola,5,U
...,...,...,...,...,...,...,...
70751,ZWGE72FL,ZWGE72FL00000396,Harare,Harare,Zimbabwe,396,U
70752,ZWGE72FL,ZWGE72FL00000397,Makoni,Manicaland,Zimbabwe,397,R
70753,ZWGE72FL,ZWGE72FL00000398,Hurungwe,Mashonaland West,Zimbabwe,398,R
70754,ZWGE72FL,ZWGE72FL00000399,Harare,Harare,Zimbabwe,399,U


In [227]:
df = pd.read_pickle(preprocessing_f)
print(len(df['generated clnr from case id'].dropna()), len(df['generated hhnr from case id'].dropna()))
#these are empty from steps before
df = df.drop(columns=['generated clnr from case id', 'generated hhnr from case id'])

12110 84200


In [228]:
df = df.merge(locations_df, on='GEID', how='inner')
df[df['cluster number'] != df['DHSCLUST']][['GEID', 'adm2_gaul', 'adm1_gaul', 'adm0_gaul', 'DHSCLUST', 'cluster number']]

,GEID,adm2_gaul,adm1_gaul,adm0_gaul,DHSCLUST,cluster number
455349,GHGE33FL00000082,Agona West,Central,Ghana,81,82
455350,GHGE33FL00000082,Agona West,Central,Ghana,81,82
455351,GHGE33FL00000082,Agona West,Central,Ghana,81,82
455352,GHGE33FL00000082,Agona West,Central,Ghana,81,82
455353,GHGE33FL00000082,Agona West,Central,Ghana,81,82
...,...,...,...,...,...,...
1479154,UGGE61FL00000132,Central Kampala,Kampala,Uganda,63,132
1479155,UGGE61FL00000132,Central Kampala,Kampala,Uganda,63,132
1479156,UGGE61FL00000132,Central Kampala,Kampala,Uganda,63,132
1479157,UGGE61FL00000132,Central Kampala,Kampala,Uganda,63,132


In [229]:
df[['type of place of residence', 'URBAN_RURA']].value_counts(normalize=True, dropna=False) * 100


type of place of residence  URBAN_RURA
rural                       R             65.062716
urban                       U             34.874341
rural                       U              0.050168
urban                       R              0.012775
Name: proportion, dtype: float64

In [230]:
df2 = df[~df['cluster number'] != df['DHSCLUST']]
print(len(df))
df2

1612581


,country code and phase,GEID,DHSID,DHSID + HHID,version_nr,subversion_nr,cluster number,household number,number of household members,month of interview,...,"owns livestock, herds or farm animals",owns chickens/poultry,country code,number of households sharing toilet: applicable,GEID_init,adm2_gaul,adm1_gaul,adm0_gaul,DHSCLUST,URBAN_RURA
0,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000001,5,1,1,1,3.0,12.0,...,NaN,NaN,AO,applicable,AOGE52FL,Lobito,Benguela,Angola,1,U
1,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000002,5,1,1,2,3.0,12.0,...,NaN,NaN,AO,applicable,AOGE52FL,Lobito,Benguela,Angola,1,U
2,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000003,5,1,1,3,3.0,12.0,...,NaN,NaN,AO,applicable,AOGE52FL,Lobito,Benguela,Angola,1,U
3,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000004,5,1,1,4,5.0,12.0,...,NaN,NaN,AO,applicable,AOGE52FL,Lobito,Benguela,Angola,1,U
4,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000005,5,1,1,5,3.0,12.0,...,NaN,NaN,AO,applicable,AOGE52FL,Lobito,Benguela,Angola,1,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1612576,ZW7,ZWGE72FL00000400,ZWHR72SV00000400,ZWHR72SV0000040000000025,7,2,400,25,5.0,11.0,...,yes,31.0,ZW,applicable,ZWGE72FL,Murehwa,Mashonaland East,Zimbabwe,400,R
1612577,ZW7,ZWGE72FL00000400,ZWHR72SV00000400,ZWHR72SV0000040000000026,7,2,400,26,2.0,11.0,...,yes,10.0,ZW,not applicable,ZWGE72FL,Murehwa,Mashonaland East,Zimbabwe,400,R
1612578,ZW7,ZWGE72FL00000400,ZWHR72SV00000400,ZWHR72SV0000040000000027,7,2,400,27,5.0,11.0,...,yes,0,ZW,applicable,ZWGE72FL,Murehwa,Mashonaland East,Zimbabwe,400,R
1612579,ZW7,ZWGE72FL00000400,ZWHR72SV00000400,ZWHR72SV0000040000000028,7,2,400,28,8.0,11.0,...,yes,5.0,ZW,applicable,ZWGE72FL,Murehwa,Mashonaland East,Zimbabwe,400,R


In [231]:
### Splitting numeric and string columns as well as meta data columns
df = df2

df = df[(df['year of interview'].notna())
        & (df['month of interview'].notna()) & (df['day of interview'].notna())]
# Initialize empty lists to hold the names of numeric and categorical columns
sub_df = df[['GEID', 'household number'] + [c[6:] for c in group_by_cols]]
sub_df.rename(columns={c: 'Meta; ' + c for c in sub_df.columns}, inplace=True)
numeric_columns = [sub_df.copy()]
categorical_columns = [sub_df.copy()]
meta_data_cols = ['country code and phase', 'GEID', 'DHSID', 'DHSID + HHID', 'version_nr', 'subversion_nr', 'household number', 'case identification', 'DHSCLUST', 'country code', 
       'adm1_gaul', 'adm0_gaul', 'GEID_init', 'month of interview',
       'year of interview', 'day of interview', 'cluster number', 'adm2_gaul', 'URBAN_RURA'
    #    'generated hhnr from case id',
    #    'generated clnr from case id', 
       ]
for group_by_col in group_by_cols:
    if group_by_col not in meta_data_cols and group_by_col[6:] not in meta_data_cols:
        warn(f'group_by_col {group_by_col} not in meta_data_cols, may result in errors!')
        meta_data_cols.append(group_by_col)
    
# Iterate over each column
for col in df.columns:
    if col in meta_data_cols:
        continue
    found_nr = False
    # Create a new column for the numeric values
    nrcol = pd.to_numeric(df[col], errors='coerce')
    if nrcol.notna().sum() > 0:
        found_nr = True
        #remove integer artefacts
        if col not in ['type of cooking fuel', 'language of interview', 'owns land usable for agriculture']:
            #rename col
            nrcol.name = 'DHS Num; ' + col
            numeric_columns.append(nrcol)
    # Create a new column for the non-numeric values
    str_col = df[col].where(nrcol.isna())
    if str_col.notna().sum() > 0:
        #only "don't know" or similar
        if col != 'number of rooms used for sleeping' and col != 'hectares of agricultural land (1 decimal)':
            #rename col
            str_col.name = 'DHS Cat; ' + col
            categorical_columns.append(str_col)
        if found_nr:
            print('found both', col)
            print(df[col].value_counts(dropna=False, normalize=True) * 100)
            print(str_col.value_counts(dropna=False, normalize=True) * 100)


/tmp/ipykernel_3654222/4149456488.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df.rename(columns={c: 'Meta; ' + c for c in sub_df.columns}, inplace=True)


found both language of interview
language of interview
NaN         44.564530
english      5.852616
chichewa     4.643598
hausa        3.561338
french       2.926806
              ...    
arabic       0.000062
43.0         0.000062
85.0         0.000062
sherbro      0.000062
yalunka      0.000062
Name: proportion, Length: 239, dtype: float64
DHS Cat; language of interview
NaN                   44.934149
english                5.852616
chichewa               4.643598
hausa                  3.561338
french                 2.926806
                        ...    
arabic                 0.000062
yalunka                0.000062
tamacheck/bella        0.000062
hararigna/adirigna     0.000062
sherbro                0.000062
Name: proportion, Length: 212, dtype: float64
found both type of cooking fuel
type of cooking fuel
wood                       49.062871
charcoal                   14.069262
firewood, straw            10.082370
NaN                         9.608978
lpg                        

In [232]:
### Create meta data, numeric and categorical dfs
meta_data_df = df[meta_data_cols].copy()
# Create numeric df
numeric_df = pd.concat(numeric_columns, axis=1)

# Create categorical df
categorical_df = pd.concat(categorical_columns, axis=1)

assert (meta_data_df.index == numeric_df.index).all() & (numeric_df.index == categorical_df.index).all()
if urban_rural_all_mode == 'all':
    pass
elif urban_rural_all_mode == 'U':
    meta_data_df = meta_data_df[meta_data_df['URBAN_RURA'] == 'U']
    # Adjust numeric and categorical dfs to match the index of meta_data_df
    numeric_df = numeric_df.loc[meta_data_df.index]
    categorical_df = categorical_df.loc[meta_data_df.index]
elif urban_rural_all_mode == 'R':
    meta_data_df = meta_data_df[meta_data_df['URBAN_RURA'] == 'R']
    # Adjust numeric and categorical dfs to match the index of meta_data_df
    numeric_df = numeric_df.loc[meta_data_df.index]
    categorical_df = categorical_df.loc[meta_data_df.index]
else:
    raise ValueError(f"Wrong urban_rural_mode {urban_rural_all_mode}")

assert (meta_data_df.index == numeric_df.index).all() & (numeric_df.index == categorical_df.index).all()

### Generate date and co
meta_data_df['year'] = meta_data_df['year of interview'].astype(float).astype('Int32')
meta_data_df['month'] = meta_data_df['month of interview'].astype(float).astype('Int32')
meta_data_df['day'] = meta_data_df['day of interview'].astype(float).astype('Int32')

#replace days >28 with 28
meta_data_df.loc[meta_data_df['day'] > 28, 'day'] = 28

# Now you can combine the year, month, and day columns into a single date column
meta_data_df['date'] = pd.to_datetime(meta_data_df[['year', 'month', 'day']])

# Group the DataFrame by 'GEID_init' and 'adm2_gaul'/'cluster number'
grouped = meta_data_df.groupby([c[6:] for c in group_by_cols])

# Calculate the median date for each group
median_date = grouped['date'].median()

# Rename the date column in median_date
median_date.rename('median_date', inplace=True)

# Merge meta_data_df with median_date
meta_data_df = meta_data_df.join(median_date, on=[c[6:] for c in group_by_cols])

# Convert the median date to separate year, month, and day columns
meta_data_df['year'] = meta_data_df['median_date'].dt.year.astype('Int32')
meta_data_df['month'] = meta_data_df['median_date'].dt.month.astype('Int32')

assert (meta_data_df.index == numeric_df.index).all() & (numeric_df.index == categorical_df.index).all()

cols = [c for c in meta_data_df.columns if 'URBAN_RURA' in c]
meta_data_df[cols].value_counts(dropna=False)

URBAN_RURA
R             1045338
U              560098
Name: count, dtype: int64

In [233]:
# Drop the original date column
meta_data_df['date'] = meta_data_df['median_date']
meta_data_df.drop(columns=['day', 'year of interview', 'month of interview', 'day of interview', 'median_date'], inplace=True)

def rreplace(s, old, new, occurrence=1):
    li = s.rsplit(old, occurrence)
    return new.join(li)

### to do: urban/rural mode
#to do: this should be created after grouping!? Maybe it makes sense even before grouping??? 
# -> yes prolly, as it increases the continuity of the distribution (more HH than adm2) (these are not comparable if compared for urban/rural modes)
for col in one_hot_cols:
    print(col)
    # Convert the column to a one-hot encoded DataFrame
    one_hot = pd.get_dummies(meta_data_df[col], prefix=col)

    # Replace '_' with ': ' in column names
    one_hot.columns = ['Meta one-hot encoding; ' + rreplace(col, '_', ': ', 1) for col in one_hot.columns]

    # Add the one-hot encoded columns to meta_data_df
    meta_data_df = pd.concat([meta_data_df, one_hot], axis=1)
    
for col in frequency_hot_cols:
    # Calculate the frequency of each category in the column
    freq = meta_data_df[col].value_counts(normalize=True)

    # Map the frequencies to the original DataFrame
    meta_data_df['Meta frequency encoding; ' + col] = meta_data_df[col].map(freq)

# rename columns
meta_data_df.rename(columns={c: 'Meta; ' + c for c in meta_data_df.columns if c[:4] != 'Meta'}, inplace=True)
cols = [c for c in meta_data_df.columns if 'URBAN_RURA' in c]
print(meta_data_df[cols].value_counts())
print(meta_data_df.index)
assert (meta_data_df.index == numeric_df.index).all() & (numeric_df.index == categorical_df.index).all()

meta_data_df[cols]

adm0_gaul
Meta; URBAN_RURA
R                   1045338
U                    560098
Name: count, dtype: int64
Index([      0,       1,       2,       3,       4,       5,       6,       7,
             8,       9,
       ...
       1612571, 1612572, 1612573, 1612574, 1612575, 1612576, 1612577, 1612578,
       1612579, 1612580],
      dtype='int64', length=1605436)


,Meta; URBAN_RURA
0,U
1,U
2,U
3,U
4,U
...,...
1612576,R
1612577,R
1612578,R
1612579,R


In [234]:
for col in categorical_df.columns:
    print(categorical_df[col].dtype)
    print(categorical_df[col].value_counts(normalize=True, dropna=False) * 100)
    print('\n')

object
Meta; GEID
EGGE32FL00670122    0.007350
EGGE42FL00850329    0.007163
EGGE61FL08400151    0.006727
EGGE42FL00550211    0.005481
EGGE61FL08400116    0.005419
                      ...   
EGGE4BFL00990102    0.000062
EGGE4BFL04130101    0.000062
EGGE61FL01234204    0.000062
NIGE81FL00000009    0.000062
EGGE4BFL00962401    0.000062
Name: proportion, Length: 67316, dtype: float64


int64
Meta; household number
1        1.990986
3        1.952180
5        1.931376
7        1.930317
8        1.916053
           ...   
18625    0.000062
19526    0.000062
10517    0.000062
15122    0.000062
6945     0.000062
Name: proportion, Length: 14803, dtype: float64


object
Meta; GEID_init
NGGE7BFL    2.505238
NGGE6AFL    2.380350
KEGE8AFL    2.361415
KEGE71FL    2.256334
NGGE52FL    2.122165
              ...   
MWGE71FL    0.212092
MWGE6AFL    0.212030
AOGE52FL    0.161887
CIGE3BFL    0.132176
MLGE63FL    0.098291
Name: proportion, Length: 155, dtype: float64


object
Meta; adm2_gaul
Bamako     

In [235]:
for col in numeric_df.columns:
    print(numeric_df[col].dtype)
    print(numeric_df[col].value_counts(normalize=True, dropna=False) * 100)
    print('\n')

object
Meta; GEID
EGGE32FL00670122    0.007350
EGGE42FL00850329    0.007163
EGGE61FL08400151    0.006727
EGGE42FL00550211    0.005481
EGGE61FL08400116    0.005419
                      ...   
EGGE4BFL00990102    0.000062
EGGE4BFL04130101    0.000062
EGGE61FL01234204    0.000062
NIGE81FL00000009    0.000062
EGGE4BFL00962401    0.000062
Name: proportion, Length: 67316, dtype: float64


int64
Meta; household number
1        1.990986
3        1.952180
5        1.931376
7        1.930317
8        1.916053
           ...   
18625    0.000062
19526    0.000062
10517    0.000062
15122    0.000062
6945     0.000062
Name: proportion, Length: 14803, dtype: float64


object
Meta; GEID_init
NGGE7BFL    2.505238
NGGE6AFL    2.380350
KEGE8AFL    2.361415
KEGE71FL    2.256334
NGGE52FL    2.122165
              ...   
MWGE71FL    0.212092
MWGE6AFL    0.212030
AOGE52FL    0.161887
CIGE3BFL    0.132176
MLGE63FL    0.098291
Name: proportion, Length: 155, dtype: float64


object
Meta; adm2_gaul
Bamako     

In [236]:
for col in meta_data_df.columns:
    # print(col)
    # print(meta_data_df[col])
    # print(meta_data_df[col].dtype)
    print(meta_data_df[col].value_counts(normalize=True, dropna=False) * 100)
    print('\n')
meta_data_df.to_csv(meta_data_cols_out_f.replace('.pkl', '.csv'), index=False)
meta_data_df.to_pickle(meta_data_cols_out_f)

Meta; country code and phase
NG6    3.224046
EG4    2.914660
KE6    2.660025
NG7    2.505238
KE8    2.361415
         ...   
SZ5    0.296244
NI7    0.294562
BJ3    0.280235
KM6    0.267591
TZ3    0.221871
Name: proportion, Length: 117, dtype: float64


Meta; GEID
EGGE32FL00670122    0.007350
EGGE42FL00850329    0.007163
EGGE61FL08400151    0.006727
EGGE42FL00550211    0.005481
EGGE61FL08400116    0.005419
                      ...   
EGGE4BFL00990102    0.000062
EGGE4BFL04130101    0.000062
EGGE61FL01234204    0.000062
NIGE81FL00000009    0.000062
EGGE4BFL00962401    0.000062
Name: proportion, Length: 67316, dtype: float64


Meta; DHSID
EGHR33SV00670122    0.007350
EGHR42SV00850329    0.007163
EGHR61SV08400151    0.006727
EGHR42SV00550211    0.005481
EGHR61SV08400116    0.005419
                      ...   
EGHR4ASV00990102    0.000062
EGHR4ASV04130101    0.000062
EGHR61SV01234204    0.000062
NIHR82SV00000009    0.000062
EGHR4ASV00962401    0.000062
Name: proportion, Length: 67316, dty

In [237]:
print(len(categorical_df), len(df), len(numeric_df), len(meta_data_df))
# print('GEID' in categorical_df.columns, 'GEID' in numeric_df.columns, 'GEID' in meta_data_df.columns)
# print('household number' in categorical_df.columns, 'household number' in numeric_df.columns, 'household number' in meta_data_df.columns)

1605436 1605436 1605436 1605436


In [238]:
cols = ['GEID', 'adm2_gaul', 'adm1_gaul', 'adm0_gaul', 'year', 'month', 'household number', 'cluster number', 'date', 'URBAN_RURA']
cols = ['Meta; ' + c for c in cols]

for c in group_by_cols:
    if c in cols:    
        cols.remove(c)

for col in meta_data_df.columns:
    if ' applicable' in col or 'one-hot encoding' in col or 'frequency encoding' in col:
        print(col)
        cols.append(col)

assert (meta_data_df.index == numeric_df.index).all() & (numeric_df.index == categorical_df.index).all()

numeric_df2 = pd.concat([meta_data_df[cols], numeric_df], axis=1)

assert meta_data_df.index.all() == numeric_df2.index.all() == categorical_df.index.all()

#only working when not dropping due to urban/rural mode
if urban_rural_all_mode == 'all':
    assert len(df) == len(numeric_df2) == len(meta_data_df)

for na, df in [('numeric', numeric_df2), ('cat', categorical_df)]:
    for col in df.columns:
        if '_x' in col:
            print(na, col)

numeric_df2

Meta one-hot encoding; adm0_gaul: Angola
Meta one-hot encoding; adm0_gaul: Benin
Meta one-hot encoding; adm0_gaul: Burkina Faso
Meta one-hot encoding; adm0_gaul: Burundi
Meta one-hot encoding; adm0_gaul: Cameroon
Meta one-hot encoding; adm0_gaul: Central African Republic
Meta one-hot encoding; adm0_gaul: Chad
Meta one-hot encoding; adm0_gaul: Comoros
Meta one-hot encoding; adm0_gaul: Côte d'Ivoire
Meta one-hot encoding; adm0_gaul: Democratic Republic of the Congo
Meta one-hot encoding; adm0_gaul: Egypt
Meta one-hot encoding; adm0_gaul: Ethiopia
Meta one-hot encoding; adm0_gaul: Gabon
Meta one-hot encoding; adm0_gaul: Gambia
Meta one-hot encoding; adm0_gaul: Ghana
Meta one-hot encoding; adm0_gaul: Guinea
Meta one-hot encoding; adm0_gaul: Kenya
Meta one-hot encoding; adm0_gaul: Lesotho
Meta one-hot encoding; adm0_gaul: Liberia
Meta one-hot encoding; adm0_gaul: Madagascar
Meta one-hot encoding; adm0_gaul: Malawi
Meta one-hot encoding; adm0_gaul: Mali
Meta one-hot encoding; adm0_gaul: Moro

,Meta; GEID,Meta; adm1_gaul,Meta; adm0_gaul,Meta; year,Meta; month,Meta; household number,Meta; cluster number,Meta; date,Meta; URBAN_RURA,Meta one-hot encoding; adm0_gaul: Angola,...,DHS Num; number of eligible men in hh,DHS Num; number of children 5 and under (de jure),DHS Num; number of households sharing toilet,DHS Num; hectares of agricultural land (1 decimal),DHS Num; number of rooms used for sleeping,DHS Num; owns goats,DHS Num; owns horses/ donkeys/ mules,DHS Num; time to get to water source (minutes),DHS Num; owns cows/ bulls,DHS Num; owns chickens/poultry
0,AOGE52FL00000001,Benguela,Angola,2006,12,1,1,2006-12-01,U,True,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,AOGE52FL00000001,Benguela,Angola,2006,12,2,1,2006-12-01,U,True,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,AOGE52FL00000001,Benguela,Angola,2006,12,3,1,2006-12-01,U,True,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
3,AOGE52FL00000001,Benguela,Angola,2006,12,4,1,2006-12-01,U,True,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,AOGE52FL00000001,Benguela,Angola,2006,12,5,1,2006-12-01,U,True,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1612576,ZWGE72FL00000400,Mashonaland East,Zimbabwe,2015,10,25,400,2015-10-27,R,False,...,NaN,1.0,0.0,0.0,2.0,7.0,0.0,0.0,NaN,31.0
1612577,ZWGE72FL00000400,Mashonaland East,Zimbabwe,2015,10,26,400,2015-10-27,R,False,...,NaN,0.0,10.0,30.0,2.0,0.0,0.0,0.0,NaN,10.0
1612578,ZWGE72FL00000400,Mashonaland East,Zimbabwe,2015,10,27,400,2015-10-27,R,False,...,NaN,2.0,0.0,10.0,3.0,3.0,0.0,10.0,NaN,0.0
1612579,ZWGE72FL00000400,Mashonaland East,Zimbabwe,2015,10,28,400,2015-10-27,R,False,...,NaN,0.0,0.0,12.0,3.0,8.0,0.0,0.0,NaN,5.0


In [239]:
for col in numeric_df2:
    print(col)

Meta; GEID
Meta; adm1_gaul
Meta; adm0_gaul
Meta; year
Meta; month
Meta; household number
Meta; cluster number
Meta; date
Meta; URBAN_RURA
Meta one-hot encoding; adm0_gaul: Angola
Meta one-hot encoding; adm0_gaul: Benin
Meta one-hot encoding; adm0_gaul: Burkina Faso
Meta one-hot encoding; adm0_gaul: Burundi
Meta one-hot encoding; adm0_gaul: Cameroon
Meta one-hot encoding; adm0_gaul: Central African Republic
Meta one-hot encoding; adm0_gaul: Chad
Meta one-hot encoding; adm0_gaul: Comoros
Meta one-hot encoding; adm0_gaul: Côte d'Ivoire
Meta one-hot encoding; adm0_gaul: Democratic Republic of the Congo
Meta one-hot encoding; adm0_gaul: Egypt
Meta one-hot encoding; adm0_gaul: Ethiopia
Meta one-hot encoding; adm0_gaul: Gabon
Meta one-hot encoding; adm0_gaul: Gambia
Meta one-hot encoding; adm0_gaul: Ghana
Meta one-hot encoding; adm0_gaul: Guinea
Meta one-hot encoding; adm0_gaul: Kenya
Meta one-hot encoding; adm0_gaul: Lesotho
Meta one-hot encoding; adm0_gaul: Liberia
Meta one-hot encoding; ad

In [240]:
### Aggregating the data: (categorical values are aggregated further below)
# mean, median, std, skewness, kurtosis, percentage of valid answers for numerical columns
# mean for one-hot and frequency encoding columns //should be the same as .agg('first')?!
# first for meta data columns except encoding columns
numeric_df = numeric_df2
print(meta_data_df.index)
print(categorical_df.index)
print(numeric_df.index)
assert (meta_data_df.index == numeric_df.index).all() & (numeric_df.index == categorical_df.index).all()

# Define the columns to include in each type of aggregation
cols_for_stats = [col for col in numeric_df.columns if 'DHS Num;' in col]
cols_for_mean = [col for col in numeric_df.columns if 'Meta one-hot encoding;' in col or 'Meta frequency encoding;' in col or 'Meta DHS applicable;' in col]
cols_for_first = ['Meta; ' + c for c in ['adm1_gaul', 'adm0_gaul', 'year', 'month', 'date']]
if 'Meta; adm2_gaul' != group_by_cols[1]:
    cols_for_first.append('Meta; adm2_gaul')

# Group the DataFrame
grouped = numeric_df.groupby(group_by_cols)

# Perform the aggregations
agg_df = grouped[cols_for_stats].agg(['mean', 'median', 'std', 
                                      lambda x: scipy.stats.skew(x.dropna()) if not x.dropna().empty else np.nan,
                                      lambda x: scipy.stats.kurtosis(x.dropna()) if not x.dropna().empty else np.nan])
agg_df.rename(columns={'<lambda_0>': 'skewness', '<lambda_1>': 'kurtosis'}, level=1, inplace=True)


mean_df = grouped[cols_for_mean].agg('mean')
first_df = grouped[cols_for_first].agg('first')

# Rename the columns of agg_df
agg_df.columns = [f'{col[0]}: {col[1]}' for col in agg_df.columns]

# Combine the results into a single DataFrame
result = pd.concat([first_df, mean_df, agg_df], axis=1)

# Calculate the count of non-NaN values and the size of each group
count_df = grouped[cols_for_stats].count()
group_sizes = grouped.size()

# Calculate the percentage of valid answers
percentage_df = count_df.div(group_sizes, axis=0) * 100

# Add the group sizes and percentages to the result DataFrame
result['Meta; amount of households'] = group_sizes
for col in percentage_df.columns:
    result[f'Meta; {col}: percentage of valid answers'] = percentage_df[col]

del numeric_df2, mean_df, first_df, agg_df, count_df, group_sizes, percentage_df, numeric_df, grouped

numerical_grouped_df = result
numerical_grouped_df

Index([      0,       1,       2,       3,       4,       5,       6,       7,
             8,       9,
       ...
       1612571, 1612572, 1612573, 1612574, 1612575, 1612576, 1612577, 1612578,
       1612579, 1612580],
      dtype='int64', length=1605436)
Index([      0,       1,       2,       3,       4,       5,       6,       7,
             8,       9,
       ...
       1612571, 1612572, 1612573, 1612574, 1612575, 1612576, 1612577, 1612578,
       1612579, 1612580],
      dtype='int64', length=1605436)
Index([      0,       1,       2,       3,       4,       5,       6,       7,
             8,       9,
       ...
       1612571, 1612572, 1612573, 1612574, 1612575, 1612576, 1612577, 1612578,
       1612579, 1612580],
      dtype='int64', length=1605436)


Meta; adm1_gaul Meta; adm0_gaul  \
Meta; GEID_init Meta; adm2_gaul                                              
AOGE52FL        Ambaca                        Kuanza Norte          Angola   
                Amboim                          Cuanza Sul          Angola   
                Andulo                                 Bie          Angola   
                Baia Farta                        Benguela          Angola   
                Balombo                           Benguela          Angola   
...                                                    ...             ...   
ZWGE72FL        Umzingwane              Matabeleland South        Zimbabwe   
                Uzumba-Maramba-Pfungwe    Mashonaland East        Zimbabwe   
                Zaka                              Masvingo        Zimbabwe   
                Zvimba                    Mashonaland West        Zimbabwe   
                Zvishavane                        Midlands        Zimbabwe   

                                        Meta; year  Meta; month Meta; date  \
Meta; GEID_init Meta; adm2_gaul                                              
AOGE52FL        Ambaca                        2007            3 2007-03-06   
                Amboim                        2006           12 2006-12-11   
                Andulo                        2007            1 2007-01-08   
                Baia Farta                    2006           11 2006-11-27   
                Balombo                       2006           12 2006-12-07   
...                                            ...          ...        ...   
ZWGE72FL        Umzingwane                    2015            7 2015-07-10   
                Uzumba-Maramba-Pfungwe        2015           10 2015-10-12   
                Zaka                          2015           10 2015-10-19   
                Zvimba                        2015           11 2015-11-25   
                Zvishavane                    2015            7 2015-07-28   

                                        Meta one-hot encoding; adm0_gaul: Angola  \
Meta; GEID_init Meta; adm2_gaul                                                    
AOGE52FL        Ambaca                                                       1.0   
                Amboim                                                       1.0   
                Andulo                                                       1.0   
                Baia Farta                                                   1.0   
                Balombo                                                      1.0   
...                                                                          ...   
ZWGE72FL        Umzingwane                                                   0.0   
                Uzumba-Maramba-Pfungwe                                       0.0   
                Zaka                                                         0.0   
                Zvimba                                                       0.0   
                Zvishavane                                                   0.0   

                                        Meta one-hot encoding; adm0_gaul: Benin  \
Meta; GEID_init Meta; adm2_gaul                                                   
AOGE52FL        Ambaca                                                      0.0   
                Amboim                                                      0.0   
                Andulo                                                      0.0   
                Baia Farta                                                  0.0   
                Balombo                                                     0.0   
...                                                                         ...   
ZWGE72FL        Umzingwane                                                  0.0   
                Uzumba-Maramba-Pfungwe                                      0.0   
                Zaka                                                        0.0   
       

In [241]:
for col in numerical_grouped_df:
    print(col)

Meta; adm1_gaul
Meta; adm0_gaul
Meta; year
Meta; month
Meta; date
Meta one-hot encoding; adm0_gaul: Angola
Meta one-hot encoding; adm0_gaul: Benin
Meta one-hot encoding; adm0_gaul: Burkina Faso
Meta one-hot encoding; adm0_gaul: Burundi
Meta one-hot encoding; adm0_gaul: Cameroon
Meta one-hot encoding; adm0_gaul: Central African Republic
Meta one-hot encoding; adm0_gaul: Chad
Meta one-hot encoding; adm0_gaul: Comoros
Meta one-hot encoding; adm0_gaul: Côte d'Ivoire
Meta one-hot encoding; adm0_gaul: Democratic Republic of the Congo
Meta one-hot encoding; adm0_gaul: Egypt
Meta one-hot encoding; adm0_gaul: Ethiopia
Meta one-hot encoding; adm0_gaul: Gabon
Meta one-hot encoding; adm0_gaul: Gambia
Meta one-hot encoding; adm0_gaul: Ghana
Meta one-hot encoding; adm0_gaul: Guinea
Meta one-hot encoding; adm0_gaul: Kenya
Meta one-hot encoding; adm0_gaul: Lesotho
Meta one-hot encoding; adm0_gaul: Liberia
Meta one-hot encoding; adm0_gaul: Madagascar
Meta one-hot encoding; adm0_gaul: Malawi
Meta one-ho

In [242]:
cols = [c for c in meta_data_df.columns if 'URBAN_RURA' in c]
meta_data_df[cols].value_counts(dropna=False)

Meta; URBAN_RURA
R                   1045338
U                    560098
Name: count, dtype: int64

In [243]:
### Create categorical grouped df
# merge with meta data to ensure only valid rows are used in urban/rural mode
grouped_dfs = []

assert len(categorical_df) == len(meta_data_df)
# categorical_df = pd.merge(meta_data_df[group_by_cols + ['Meta; URBAN_RURA']], categorical_df, on=group_by_cols, how='inner')

assert (categorical_df.index == meta_data_df.index).all()
categorical_df['DHS Cat; URBAN_RURA'] = meta_data_df['Meta; URBAN_RURA']
# categorical_df = categorical_df.drop(columns=['Meta; URBAN_RURA'])

# Group the DataFrame by 'GEID_init' and 'adm2_gaul'
grouped = categorical_df.groupby(group_by_cols)

from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

def process_column(col):
    if col not in sub_df.columns:
        # Calculate the normalized count of each category excluding NaNs
        counts = grouped[col].value_counts(normalize=True).unstack()
  
        # Only fill NaNs with 0 if there were values in the group
        if not counts.empty:
            counts = counts.fillna(0)
  
        # Calculate the normalized count of NaNs
        nan_counts = grouped[col].apply(lambda x: x.isna().mean()).rename('NaN')

        # Append the count of NaNs to the counts of the other categories
        counts = pd.concat([counts, nan_counts], axis=1)
        
        # Flatten the MultiIndex and add a ': ' between the original column name and the value_counts.index items
        counts.columns = [f'{col}: {c}' for c in counts.columns]
        return counts
    
    
    
# Create a pool of processes
with ProcessPoolExecutor(max_workers=4) as executor:
    # Use tqdm for progress bar
    grouped_dfs = list(tqdm(executor.map(process_column, categorical_df.columns), total=len(categorical_df.columns)))

# Concatenate the results
grouped_df = pd.concat(grouped_dfs, axis=1)
categorical_grouped_df = grouped_df


100%|██████████| 67/67 [00:28<00:00,  2.38it/s]


In [244]:
# merge both dfs
grouped_df = pd.merge(numerical_grouped_df, categorical_grouped_df, on=group_by_cols, how='inner')
grouped_df

/tmp/ipykernel_3654222/1384408156.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_df = pd.merge(numerical_grouped_df, categorical_grouped_df, on=group_by_cols, how='inner')
/tmp/ipykernel_3654222/1384408156.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_df = pd.merge(numerical_grouped_df, categorical_grouped_df, on=group_by_cols, how='inner')


Meta; adm1_gaul Meta; adm0_gaul  \
Meta; GEID_init Meta; adm2_gaul                                              
AOGE52FL        Ambaca                        Kuanza Norte          Angola   
                Amboim                          Cuanza Sul          Angola   
                Andulo                                 Bie          Angola   
                Baia Farta                        Benguela          Angola   
                Balombo                           Benguela          Angola   
...                                                    ...             ...   
ZWGE72FL        Umzingwane              Matabeleland South        Zimbabwe   
                Uzumba-Maramba-Pfungwe    Mashonaland East        Zimbabwe   
                Zaka                              Masvingo        Zimbabwe   
                Zvimba                    Mashonaland West        Zimbabwe   
                Zvishavane                        Midlands        Zimbabwe   

                                        Meta; year  Meta; month Meta; date  \
Meta; GEID_init Meta; adm2_gaul                                              
AOGE52FL        Ambaca                        2007            3 2007-03-06   
                Amboim                        2006           12 2006-12-11   
                Andulo                        2007            1 2007-01-08   
                Baia Farta                    2006           11 2006-11-27   
                Balombo                       2006           12 2006-12-07   
...                                            ...          ...        ...   
ZWGE72FL        Umzingwane                    2015            7 2015-07-10   
                Uzumba-Maramba-Pfungwe        2015           10 2015-10-12   
                Zaka                          2015           10 2015-10-19   
                Zvimba                        2015           11 2015-11-25   
                Zvishavane                    2015            7 2015-07-28   

                                        Meta one-hot encoding; adm0_gaul: Angola  \
Meta; GEID_init Meta; adm2_gaul                                                    
AOGE52FL        Ambaca                                                       1.0   
                Amboim                                                       1.0   
                Andulo                                                       1.0   
                Baia Farta                                                   1.0   
                Balombo                                                      1.0   
...                                                                          ...   
ZWGE72FL        Umzingwane                                                   0.0   
                Uzumba-Maramba-Pfungwe                                       0.0   
                Zaka                                                         0.0   
                Zvimba                                                       0.0   
                Zvishavane                                                   0.0   

                                        Meta one-hot encoding; adm0_gaul: Benin  \
Meta; GEID_init Meta; adm2_gaul                                                   
AOGE52FL        Ambaca                                                      0.0   
                Amboim                                                      0.0   
                Andulo                                                      0.0   
                Baia Farta                                                  0.0   
                Balombo                                                     0.0   
...                                                                         ...   
ZWGE72FL        Umzingwane                                                  0.0   
                Uzumba-Maramba-Pfungwe                                      0.0   
                Zaka                                                        0.0   
       

In [245]:
grouped_df.to_csv(out_f.replace('.pkl', '.csv'))
grouped_df.to_pickle(out_f)

In [246]:
for col in grouped_df.columns:
    print(col)

cols = [c for c in grouped_df.columns if 'URBAN_RURA' in c]
grouped_df[cols].value_counts(dropna=False)

Meta; adm1_gaul
Meta; adm0_gaul
Meta; year
Meta; month
Meta; date
Meta one-hot encoding; adm0_gaul: Angola
Meta one-hot encoding; adm0_gaul: Benin
Meta one-hot encoding; adm0_gaul: Burkina Faso
Meta one-hot encoding; adm0_gaul: Burundi
Meta one-hot encoding; adm0_gaul: Cameroon
Meta one-hot encoding; adm0_gaul: Central African Republic
Meta one-hot encoding; adm0_gaul: Chad
Meta one-hot encoding; adm0_gaul: Comoros
Meta one-hot encoding; adm0_gaul: Côte d'Ivoire
Meta one-hot encoding; adm0_gaul: Democratic Republic of the Congo
Meta one-hot encoding; adm0_gaul: Egypt
Meta one-hot encoding; adm0_gaul: Ethiopia
Meta one-hot encoding; adm0_gaul: Gabon
Meta one-hot encoding; adm0_gaul: Gambia
Meta one-hot encoding; adm0_gaul: Ghana
Meta one-hot encoding; adm0_gaul: Guinea
Meta one-hot encoding; adm0_gaul: Kenya
Meta one-hot encoding; adm0_gaul: Lesotho
Meta one-hot encoding; adm0_gaul: Liberia
Meta one-hot encoding; adm0_gaul: Madagascar
Meta one-hot encoding; adm0_gaul: Malawi
Meta one-ho

DHS Cat; URBAN_RURA: R  DHS Cat; URBAN_RURA: U  DHS Cat; URBAN_RURA: NaN
1.000000                0.000000                0.0                         7399
0.000000                1.000000                0.0                         2289
0.500000                0.500000                0.0                          347
0.666667                0.333333                0.0                          161
0.750000                0.250000                0.0                           93
                                                                            ... 
0.582192                0.417808                0.0                            1
0.582492                0.417508                0.0                            1
0.582524                0.417476                0.0                            1
0.582609                0.417391                0.0                            1
0.670157                0.329843                0.0                            1
Name: count, Length: 3001, dtype: in